## <font color='darkblue'>N Queues Imp (1, 2)</font>
([course link1](https://www.udemy.com/course/ai-and-combinatorial-optimization-with-meta-heuristics/learn/lecture/30551348#overview), [course link2](https://www.udemy.com/course/ai-and-combinatorial-optimization-with-meta-heuristics/learn/lecture/30551352#overview))

In [110]:
from functools import cached_property
from random import choice, uniform, shuffle
from numpy.random import randint
from typing import List

N_QUEEN = 6
TOURNAMENT_SIZE = 5
MAX_FITNESS = 1.0
CHROMOSOME_LENGTH = N_QUEEN

# Chromosome
class Individual:
  """Representation of the problem."""
  
  def __init__(self, genes=None):
    if genes:
      self.genes = genes
    else:
      self.genes = list(range(CHROMOSOME_LENGTH))
      shuffle(self.genes)
    
  @cached_property
  def penality(self) -> int:
    penality = 0
    # Penality for diagnal
    for ci, ri in enumerate(self.genes):
      for i in range(1, CHROMOSOME_LENGTH-ci):
        # Check down side
        if self.genes[ci+i] == ri + i and \
           ri + i < CHROMOSOME_LENGTH:
          penality += 1
          
        # Check up side
        if self.genes[ci+i] == ri - i and \
           ri - i >= 0:
          penality += 1
          
    return penality
    
  @cached_property
  def fitness(self) -> float:
    """Calculates the fitness values of these chromosomes"""
    return 1 / (1 + self.penality)
  
  def __repr__(self):
    rows = []
    print(self.genes)
    for i in range(CHROMOSOME_LENGTH):
      row = ['.'] * CHROMOSOME_LENGTH
      try:
        row[self.genes.index(i)] = 'o'
      except:
        pass
      rows.append('|'.join(row))
      
    return '\n'.join(rows) + '\n\n'

In [77]:
indv = Individual()
print(f'{indv} with penality={indv.penality}; fitness={indv.fitness}')

[0, 3, 2, 1, 4]
o|.|.|.|.
.|.|.|o|.
.|.|o|.|.
.|o|.|.|.
.|.|.|.|o

 with penality=6; fitness=0.14285714285714285


In [82]:
class Population:
  
  def __init__(self, population_size):
    self._population_size = population_size
    self._individuals = (
      [Individual() for _ in range(self._population_size)])

  def iter_individuals(self):
    for i in self._individuals:
      yield i
    
  def get_fittest(self):
    return max(
      map(lambda i: (i, i.fitness), self._individuals),
      key=lambda t: t[1])[0]
  
  def get_fittest_elitism(self, n: int) -> List[Individual]:
    self._individuals.sort(
      key=lambda ind: ind.fitness, reverse=True)
    return self._individuals[:n]
  
  def get_size(self):
    return self._population_size
  
  def get_individual(self, index: int):
    return self._individuals[index]
  
  def save_individual(self, index: int, individual: Individual):
    self._individuals[index] = individual

In [114]:
class GeneticAlgorithmWithElitism:
  def __init__(
    self, population_size: int=100, crossover_rate: float=0.65,
    mutation_rate: float=0.1, elitism_param: int=5):
    self._population_size = population_size
    self._crossover_rate = crossover_rate
    self._mutation_rate = mutation_rate
    self._elitism_param = elitism_param
    
  def run(self):
    pop = Population(self._population_size)
    generation_counter = 0
    
    while pop.get_fittest().fitness != MAX_FITNESS:
      generation_counter += 1
      print(
        f'Generation #{generation_counter} - '
        f'fittest is:\n{pop.get_fittest()}'
        f'with fitness value {pop.get_fittest().fitness}\n\n')
      
      pop = self.evolve_population(pop)
      
    print('Solution found!')
    print(pop.get_fittest())
    
  def crossover(self, indv1, indv2):
    cross_individual = Individual()
    available_gene_set = set(range(CHROMOSOME_LENGTH))
    for i, t in enumerate(zip(indv1.genes, indv2.genes)):
      g1, g2 = t
      if g1 in available_gene_set and g2 in available_gene_set:
        selected_gene = choice([g1, g2])
        cross_individual.genes[i] = selected_gene
        available_gene_set.remove(selected_gene)
      elif g1 in available_gene_set:
        cross_individual.genes[i] = g1
        available_gene_set.remove(g1)
      elif g2 in available_gene_set:
        cross_individual.genes[i] = g2
        available_gene_set.remove(g2)
      else:
        selected_gene = choice(list(available_gene_set))
        cross_individual.genes[i] = selected_gene
        available_gene_set.remove(selected_gene)
    
    return cross_individual
    
  def evolve_population(self, population: Population) -> Population:
    """Evolves given population"""
    next_population = Population(self._population_size)
    next_population._individuals.extend(
      population.get_fittest_elitism(self._elitism_param))
    
    # Crossover (Tournament selection)
    for i in range(self._elitism_param, next_population.get_size()):
      first = self.random_selection(population)
      second = self.random_selection(population)
      next_population.save_individual(
        i, self.crossover(first, second))
      
    # Mutation
    #for individual in next_population.iter_individuals():
    #  self.mutate(individual)
      
    return next_population
  
  def mutate(self, individual):
    for index in range(CHROMOSOME_LENGTH):
      if uniform(0, 1) < self._mutation_rate:
        individual.genes[index] = randint(CHROMOSOME_LENGTH)
  
  def random_selection(self, actual_population) -> Individual:
    """Does tournament selection."""
    new_population = Population(TOURNAMENT_SIZE)
    
    for i in range(TOURNAMENT_SIZE):
      random_index = randint(actual_population.get_size())
      new_population.save_individual(
        i, actual_population.get_individual(random_index))
      
    return new_population.get_fittest()

In [115]:
algorithm = GeneticAlgorithmWithElitism(
  population_size=10)

In [116]:
algorithm.run()

[4, 2, 1, 5, 3, 0]
Generation #1 - fittest is:
.|.|.|.|.|o
.|.|o|.|.|.
.|o|.|.|.|.
.|.|.|.|o|.
o|.|.|.|.|.
.|.|.|o|.|.

with fitness value 0.3333333333333333


[1, 3, 0, 5, 2, 4]
Generation #2 - fittest is:
.|.|o|.|.|.
o|.|.|.|.|.
.|.|.|.|o|.
.|o|.|.|.|.
.|.|.|.|.|o
.|.|.|o|.|.

with fitness value 0.3333333333333333


Solution found!
[1, 3, 5, 0, 2, 4]
.|.|.|o|.|.
o|.|.|.|.|.
.|.|.|.|o|.
.|o|.|.|.|.
.|.|.|.|.|o
.|.|o|.|.|.


